In [1]:
import random

class AssetDatabase:
    """
    A database to store information about various assets, including stocks and bonds.
    Each asset is represented by a dictionary containing its market capitalization, code, and type.
    """
    def __init__(self):
        # Initialize the database with predefined assets
        self.assets = {
            "GZMT": {"market_cap": 21700, "code": "600519"},
            "NDSD": {"market_cap": 11100, "code": "300750"}, 
            "ZYCX": {"market_cap": 529.54, "code": "603986"}, 
            "ZGGW": {"market_cap": 523.27, "code": "002049"}, 
            "RJGD": {"market_cap": 45.95, "code": "003015"}, 
            "BTSY": {"market_cap": 72.87, "code": "000595"}, 
            "SZZS": {"market_cap": 0, "type": "BONDS", "code": "510760"}
        }

    def get_asset_info(self, asset_name):
        """
        Retrieves the asset information based on its name.
        The method dynamically assigns an asset type based on market capitalization.
        """
        asset = self.assets.get(asset_name, None)
        if asset:
            asset = self._assign_type_based_on_market_cap(asset)
        return asset

    def _assign_type_based_on_market_cap(self, asset):
        """
        Dynamically assigns an asset type based on its market capitalization.
        Special handling is included for bond types.
        """
        # Handle bond type separately
        if 'type' in asset and asset['type'] == 'BONDS':
            return asset

        # Assign stock types based on market capitalization
        if asset['market_cap'] > 1000:
            asset['type'] = 'LARGE_CAP_STOCKS'
        elif 100 <= asset['market_cap'] <= 1000:
            asset['type'] = 'MID_CAP_STOCKS'
        elif asset['market_cap'] < 100:
            asset['type'] = 'SMALL_CAP_STOCKS'
        return asset

    def get_random_asset_by_type(self, asset_type):
        """
        Selects a random asset from the database based on the specified asset type.
        """
        filtered_assets = [name for name, asset in self.assets.items() if self._assign_type_based_on_market_cap(asset)['type'] == asset_type]
        if filtered_assets:
            return random.choice(filtered_assets)
        return None

In [2]:
class ETFPortfolio:
    """
    Represents an ETF (Exchange-Traded Fund) portfolio, managing allocations of various assets.
    """
    def __init__(self, name, asset_db):
        """
        Initializes the ETF portfolio with a name and a reference to an asset database.
        The portfolio starts with 100% allocation in cash.
        """
        self.name = name
        self.allocations = {"CASH": 100}  # Initial allocation is all in cash.
        self.asset_db = asset_db          # Reference to the asset database for asset information.

    def set_allocation(self, asset_type, percentage):
        """
        Sets the allocation for a given asset type. Randomly selects an asset of the specified type.
        """
        asset_name = self.asset_db.get_random_asset_by_type(asset_type)
        if asset_name:
            if self._validate_allocation(percentage):
                # Allocates the specified percentage to the asset.
                self.allocations[asset_name] = self.allocations.get(asset_name, 0) + percentage
                self._update_cash_allocation()  # Adjusts the cash allocation accordingly.
        else:
            print(f"No asset found for type '{asset_type}'.")

    def update_allocation(self, asset_type, percentage):
        """
        Updates the allocation for an existing asset of the given type.
        """
        existing_assets = [asset for asset in self.allocations if self.asset_db.get_asset_info(asset) and self.asset_db.get_asset_info(asset)['type'] == asset_type]
        if existing_assets:
            asset_name = existing_assets[0]  # Assumes updating the first asset found of that type.
            if self._validate_allocation(percentage, asset_name):
                self.allocations[asset_name] = percentage
                self._update_cash_allocation()  # Adjusts the cash allocation accordingly.
        else:
            print(f"No existing asset found for type '{asset_type}' to update.")

    def _validate_allocation(self, new_percentage, asset_type=None):
        """
        Validates that the total allocation including the new percentage does not exceed 100%.
        """
        total_percentage = sum(self.allocations.values()) - self.allocations.get(asset_type, 0) - self.allocations['CASH'] + new_percentage
        if total_percentage > 100:
            print(f"Error in '{self.name}': Total allocation exceeds 100%.")
            return False
        return True

    def _update_cash_allocation(self):
        """
        Updates the allocation for cash based on the current total allocation of other assets.
        """
        total_invested = sum(self.allocations.values()) - self.allocations['CASH']
        self.allocations['CASH'] = max(0, 100 - total_invested)

In [10]:
import re
from xml.etree.ElementTree import Element, tostring

class ASTNode:
    def __init__(self, type, value=None):
        self.type = type  # Node type, e.g., 'SET', 'UPDATE', 'Portfolio'
        self.value = value  # Node value, e.g., 'myPortfolio1', 'LARGE_CAP_STOCKS'
        self.children = []  # List of child nodes

    def add_child(self, node):
        self.children.append(node)

    def to_xml(self):
        # Recursive function to convert AST to XML
        element = Element(self.type)
        if self.value:
            element.text = str(self.value)
        for child in self.children:
            element.append(child.to_xml())
        return element

class ETFDSLInterpreter:
    def __init__(self):
        self.asset_db = AssetDatabase()
        self.etf_portfolios = {}

    def interpret(self, command_str):
        # Use regular expression to more flexibly parse the command string
        pattern = r'^(SET|UPDATE) ETF (\w+) WITH (\w+) = (\d+)%$'
        match = re.match(pattern, command_str.strip())
        if not match:
            raise ValueError(f"Invalid command format: '{command_str}'.")

        action, portfolio_name, asset_type, percentage_str = match.groups()

        try:
            percentage = int(percentage_str)
        except ValueError:
            raise ValueError(f"Invalid percentage value: '{percentage_str}' in command '{command_str}'")

        # AST generation
        root = ASTNode(action)
        portfolio_node = ASTNode('Portfolio', portfolio_name)
        asset_node = ASTNode('AssetType', asset_type)
        percentage_node = ASTNode('Percentage', percentage)

        root.add_child(portfolio_node)
        root.add_child(asset_node)
        root.add_child(percentage_node)

        # Convert AST to XML
        ast_xml = tostring(root.to_xml())
        print(ast_xml.decode())

        if portfolio_name not in self.etf_portfolios:
            self.etf_portfolios[portfolio_name] = ETFPortfolio(portfolio_name, self.asset_db)

        portfolio = self.etf_portfolios[portfolio_name]

        if action == 'SET':
            portfolio.set_allocation(asset_type, percentage)
        elif action == 'UPDATE':
            portfolio.update_allocation(asset_type, percentage)

    def get_portfolio(self, name):
        return self.etf_portfolios.get(name, None)

In [11]:
class ETFDSLInterpreter:
    """
    This class interprets and executes commands based on a Domain-Specific Language (DSL) for ETF Portfolio management.
    It maintains a collection of ETF portfolios and interacts with an asset database.
    """

    def __init__(self):
        """
        Initializes the interpreter with an asset database and an empty dictionary for storing ETF portfolios.
        """
        self.asset_db = AssetDatabase()  # Instance of the AssetDatabase to access asset information.
        self.etf_portfolios = {}         # Dictionary to store named ETF portfolios.

    def interpret(self, command_str):
        """
        Interprets a DSL command string to perform ETF portfolio operations like setting or updating allocations.
        """
        tokens = command_str.split()  # Split the command string into tokens.
        action = tokens[0]            # The action to perform (SET or UPDATE).
        portfolio_name = tokens[2]    # The name of the portfolio.
        asset_type = tokens[4]        # The type of asset to allocate.
        percentage_str = tokens[6].strip('%')  # The allocation percentage as a string.
        percentage = int(percentage_str)       # Convert the percentage to an integer.

        # Create a new portfolio if it doesn't exist.
        if portfolio_name not in self.etf_portfolios:
            self.etf_portfolios[portfolio_name] = ETFPortfolio(portfolio_name, self.asset_db)

        # Retrieve the portfolio.
        portfolio = self.etf_portfolios[portfolio_name]

        # Perform the specified action.
        if action == 'SET':
            portfolio.set_allocation(asset_type, percentage)  # Set allocation for the asset type.
        elif action == 'UPDATE':
            portfolio.update_allocation(asset_type, percentage)  # Update allocation for the asset type.

    def get_portfolio(self, name):
        """
        Retrieves a portfolio by its name.
        """
        return self.etf_portfolios.get(name, None)  # Return the portfolio if it exists.

In [11]:
# Create an instance of the ETFDSLInterpreter.
interpreter = ETFDSLInterpreter()

In [12]:
# Allocate 40% to large-cap stocks in myPortfolio1.
interpreter.interpret("SET ETF myPortfolio1 WITH LARGE_CAP_STOCKS = 40%")
print("myPortfolio1:", interpreter.get_portfolio("myPortfolio1").allocations)

<SET><Portfolio>myPortfolio1</Portfolio><AssetType>LARGE_CAP_STOCKS</AssetType><Percentage>40</Percentage></SET>
myPortfolio1: {'CASH': 60, 'NDSD': 40}


In [13]:
# Allocate 60% to bonds in myPortfolio1.
interpreter.interpret("SET ETF myPortfolio1 WITH BONDS = 60%")
print("myPortfolio1:", interpreter.get_portfolio("myPortfolio1").allocations)

<SET><Portfolio>myPortfolio1</Portfolio><AssetType>BONDS</AssetType><Percentage>60</Percentage></SET>
myPortfolio1: {'CASH': 0, 'NDSD': 40, 'SZZS': 60}


In [14]:
# Change the allocation of large-cap stocks in myPortfolio1 to 30%.
interpreter.interpret("UPDATE ETF myPortfolio1 WITH LARGE_CAP_STOCKS = 30%")
print("myPortfolio1:", interpreter.get_portfolio("myPortfolio1").allocations)

<UPDATE><Portfolio>myPortfolio1</Portfolio><AssetType>LARGE_CAP_STOCKS</AssetType><Percentage>30</Percentage></UPDATE>
myPortfolio1: {'CASH': 10, 'NDSD': 30, 'SZZS': 60}


In [15]:
# Attempt to change the allocation of bonds in myPortfolio1 to 80%, which should result in an error as it exceeds 100%.
interpreter.interpret("UPDATE ETF myPortfolio1 WITH BONDS = 80%")
print("myPortfolio1:", interpreter.get_portfolio("myPortfolio1").allocations)

<UPDATE><Portfolio>myPortfolio1</Portfolio><AssetType>BONDS</AssetType><Percentage>80</Percentage></UPDATE>
Error in 'myPortfolio1': Total allocation exceeds 100%.
myPortfolio1: {'CASH': 10, 'NDSD': 30, 'SZZS': 60}
